# Extract, Transform, Load Example

In [1]:
# Author information
__author__ = "Troy Reynolds"
__email__ = "Troy.Lloyd.Reynolds@gmail.com"

In [10]:
# Libraries
# libraries
import pandas as pd
import glob
import xml.etree.ElementTree as ET
from datetime import datetime
import urllib.request
import sys
import os
import zipfile

# Extend the directory to get created functions
sys.path.insert(0, "./helper_files")

# import helper files
from etl_helper import
    

## Get the data

In [6]:
# use urllib.request to download the file instead of !wget
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip'
filename = 'datasource.zip'
urllib.request.urlretrieve(url, filename)

('datasource.zip', <http.client.HTTPMessage at 0x23318f230d0>)

In [14]:
# path varibles
path_to_zip_file = os.path.join(".", 'datasource.zip')
data_path = os.path.join(".", "data_source_files")

# unzip the file
if not os.path.exists(newpath):
    # Create a folder to hold data
    os.makedirs(newpath)
    
    # unzip files
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(data_path)

In [15]:
# set paths for target files
tmpfile = "dealership_temp.tmp"                # extracted data
logfile = "dealership_logfile.txt"             # event logs
targetfile = "dealership_transformed_data.csv" # transformed data

## Extract

In [21]:
# Extraction Functions
def extract_from_csv(file_to_process):
    df = pd.read_csv(file_to_process)
    return df

def extract_from_json(file_to_process):
    df = pd.read_json(
        file_to_process,
        lines = True
    )
    return df

def extract_from_xml(file_to_process):
    df = pd.DataFrame(
        columns = ["car_model", "year_of_manufacture", "price", "fuel"]
    )
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    
    # extract from root
    for obs in root: # obs = observation
        # get data for each feature
        car_model = obs.find("car_model").text
        year_of_manufacture = int(obs.find("year_of_manufacture").text)
        price = float(obs.find("price").text)
        fuel = obs.find("fuel").text
        
        # append each observation top teh dataframe
        obs_data = {
            "car_model": car_model,
            "year_of_manufacture": year_of_manufacture,
            "price": price,
            "fuel": fuel
        }
        df = df.append(obs_data, ignore_index = True)
    return df

In [26]:
# combine csv, json, xml into one extract function
# NOTE: glob allows function to select all files of single data type
def extract():
    # single dataframe to hold all data
    extracted_data = pd.DataFrame(
        columns = ["car_model", "year_of_manufacture", "price", "fuel"]
    )
    
    # for csv files
    for csvfile in glob.glob(os.path.join(data_path, "*.csv")):
        extracted_data = extracted_data.append(extract_from_csv(csvfile))
        
    # for json files
    for jsonfile in glob.glob(os.path.join(data_path, "*.json")):
        extracted_data = extracted_data.append(extract_from_json(jsonfile))
    
    
    # for xml files
    for xmlfile in glob.glob(os.path.join(data_path, "*.xml")):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile))
    return extracted_data

In [27]:
extract()

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.000000,Petrol
1,sx4,2013,7089.552239,Diesel
2,ciaz,2017,10820.895522,Petrol
3,wagon r,2011,4253.731343,Petrol
4,swift,2014,6865.671642,Diesel
...,...,...,...,...
5,camry,2006,3731.343284,Petrol
6,land cruiser,2010,52238.805970,Diesel
7,corolla altis,2012,8805.970149,Petrol
8,etios liva,2013,5149.253731,Petrol


## Transform

In [28]:
# transform the variables to the desired format
def transform(data):
    data["price"] = round(data.price, 2)
    return data

## Load

In [29]:
# load function
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

In [31]:
## Log Creation

In [38]:
# log function
def log(message):
    # Hour-Minute-Second_MonthName-Day-Year
    timestamp_format = '%H:%M:%S-%h-%d-%Y'
    
    # get current timestamp
    now = datetime.now()
    
    timestamp = now.strftime(timestamp_format)
    
    # create the log file if not exist and append
    # note "a": append
    with open("dealership_logfile.txt", "a") as f: f.write(timestamp + ',' + message + '\n')

In [31]:
# log start
log("ETL Job Started")

In [32]:
# log extract step
log("Extract Phase Started")
extracted_data = extract()
log("Extract Phase Ended")

In [33]:
# log Transform Phase
log("Transformation Phase Started")
transformed_data = transform(extracted_data)
log("Transform Phase Ended")

In [34]:
# Log Load Phase
log("Load Phase Started")
load(targetfile, transformed_data)
log("Load Phase Ended")

In [35]:
# Log ETL Job Ended
log("ETL Job Ended")

In [39]:
## ETL Full Function
def ETL():
    # set paths for target files
    tmpfile = "dealership_temp.tmp"                # extracted data
    logfile = "dealership_logfile.txt"             # event logs
    targetfile = "dealership_transformed_data.csv" # transformed data
    
    # Begin ETL
    log("ETL Job Started")
    
    # Extract
    log("Extract Phase Started")
    extracted_data = extract()
    log("Extract Phase Ended")
    
    # Transform
    log("Transformation Phase Started")
    transformed_data = transform(extracted_data)
    log("Transform Phase Ended")    
    
    # Load
    log("Load Phase Started")
    load(targetfile, transformed_data)
    log("Load Phase Ended")
    
    # Finish ETL
    log("ETL Job Ended")

In [40]:
# main
ETL()